# Assignment

In this assignment we will create a model for segmentation of tumor from abdominal CT images using custom loss function modifications to increase prediction sensitivity.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 23.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### faiss library

To facilitate fast kmeans clustering, we will use an efficient algorithm implemented by the Facebook AI Research team as part of the `faiss` library. In brief, `faiss` is a library for efficient similarity search and clustering of dense vectors. More information can be found here: https://github.com/facebookresearch/faiss.

In [2]:
# --- Install faiss
% pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 4.3 MB/s 


### Imports

Use the following lines to import any additional needed libraries:

In [3]:
import numpy as np, pandas as pd
from scipy import ndimage
import tensorflow as tf
from tensorflow.keras import Input, Model, models, losses, metrics, layers, optimizers
import faiss
from jarvis.train import datasets
from jarvis.utils import io
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about he KiTS Challenge can be found here: https://kits21.kits-challenge.org/.

In [4]:
# --- Download dataset
datasets.download(name='ct/kits')

[ 2022-05-26 22:10:46 ] [====================] 100.000% : Extracting archive (0000818 / 0000818) 

{'code': '/data/raw/ct_kits', 'data': '/data/raw/ct_kits'}

### Data loader

In this assignment, only the middle 2D slice of each volume will be used to promote fast model convergence. Since this small dataset fits easily into RAM memory, the following code block may be used to load these slices into a single Numpy array.

In [38]:
def load_data(label=1, flip=True, a_min=-128, a_max=256):

    # --- Create data client
    _, _, client = datasets.prepare(name='ct/kits', keyword='3d')

    dats, lbls = [], []

    for sid, fnames, header in client.db.cursor():

        lbl, _ = io.load(fnames['lbl-crp'])
        
        if label in lbl:
            
            dat, _ = io.load(fnames['dat-crp'])
            dats.append(dat[48:49])
            lbls.append(lbl[48:49] >= label)

            if header['cohort-left'] and flip:
                dats[-1]= dats[-1][..., ::-1, :]
                lbls[-1]= lbls[-1][..., ::-1, :]

    dats = np.stack(dats, axis=0)
    lbls = np.stack(lbls, axis=0)
    
    # --- Nomralize dats
    dats = (dats - a_min) / (a_max - a_min)
    dats = dats.clip(min=0, max=1)

    return {'dat': dats, 'lbl': lbls}

Use the following cell to load all data into the `xs` dictionary:

In [39]:
# --- Load data
xs = load_data()

[ 2022-05-26 22:27:06 ] [====================] 100.000% : Iterating | 000402    

# Clusters

To create useful clusters for semantic segmentation, consider the following potential features:

* pixel (voxel) value
* pixel (voxel) coordinate location
* CNN-derived features from algorithm training

The following block can be used to create a pixel-wise (voxel-wise) feature vector based on various permuatations.

In [40]:
def create_features(x, x_weight=1, x_blur=3, coords_weight=1., backbone=None, backbone_weight=1., **kwargs):
    """
    Method to construct feature vector for clustering
    
    """
    x_ = [] 

    if x_weight > 0:
        xx = x.copy()
        if x_blur > 0:
            xx[:, 0] = ndimage.gaussian_filter(xx[:, 0], sigma=(0, x_blur, x_blur, 0))
        x_.append(xx * x_weight)

    if coords_weight > 0:
        ij = np.meshgrid(*tuple([np.linspace(0, 1, 96) for _ in range(2)]), indexing='ij')
        ij = np.expand_dims(np.stack(ij, axis=-1), axis=0)
        ij = np.stack([ij] * x.shape[0], axis=0)
        x_.append(ij * coords_weight)

    if backbone is not None:
        yy = backbone.predict(x)
        x_.append(yy * backbone_weight)

    return np.concatenate(x_, axis=-1).reshape(x.size, -1)

Based on the chosen feature feature combination, perform clustering using the `faiss` library. The following method creates a total of `n_clusters` from the input data: 

In [41]:
def create_clusters(x, n_clusters=8, **kwargs):

    x_ = create_features(x=x, **kwargs)

    kmeans = faiss.Kmeans(x_.shape[-1], n_clusters)
    kmeans.train(x_.astype('float32'))
    clusters = kmeans.index.search(x_.astype('float32'), 1)[1].reshape(x.shape)

    return kmeans, clusters

**Note**: These code blocks for clustering do not need to be modified for this assignment.

# Training

### Define the backbone model

Use the following cell block to define your backbone for the semantic segmentation task:

In [42]:
#clustering

kmeans, clusters = create_clusters(x = xs['dat'], n_clusters = 8)

In [ ]:
clusters

In [44]:
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'}

conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

concat = lambda a, b : layers.Concatenate()([a, b])

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))

In [45]:
def create_unet():
    
    x = Input(shape=(None, 96, 96, 1), dtype='float32')

    l1 = conv1(8, x)
    l2 = conv1(16, conv2(16, l1))
    l3 = conv1(32, conv2(32, l2))
    l4 = conv1(48, conv2(48, l3))
    l5 = conv1(64, conv2(64, l4))

    l6  = tran2(48, l5)
    l7  = tran2(32, conv1(48, concat(l4, l6)))
    l8  = tran2(16, conv1(32, concat(l3, l7)))
    l9  = tran2(8,  conv1(16, concat(l2, l8)))
    l10 = conv1(8,  l9)

    outputs = layers.Conv3D(filters=8, **kwargs)(l10)

    backbone = Model(inputs=x, outputs=outputs) 
    
    return backbone

### Define shared methods

Use the following cell block to define your shared methods for the pretraining and fine-tuning models. Consider the following shared components may be defined:

* generic method for creating algorithn inputs
* generic method for compiling model (including losses and metrics)

In [46]:
def create_inputs(use_augmentation=True, **kwargs):
    """
    Method to create generic model inputs (for pretraining and fine-tuning)
    
    """
    x = Input(shape=(None, 96, 96, 1))
    y = Input(shape=(None, 96, 96, 1))

    inputs = {'x': x, 'y': y}

    if use_augmentation:
        
        a = layers.Concatenate()((inputs['x'][:, 0], inputs['y'][:, 0]))
        a = layers.experimental.preprocessing.RandomRotation(factor=0.2, interpolation='nearest')(a)
        a = layers.experimental.preprocessing.RandomTranslation(0.2, 0.2, interpolation='nearest')(a)
        a = layers.experimental.preprocessing.RandomZoom(0.2, interpolation='nearest')(a)
        a = tf.expand_dims(a, axis=1)

        x = a[..., 0:1]
        y = a[..., 1:2]
        
    return inputs, x, y

In [47]:
def calculate_dsc(y_true, y_pred, cls=1):
    """ 
    Method to calculate Dice score for given class
    
    """
    true = y_true[..., 0] == cls
    pred = tf.math.argmax(y_pred, axis=-1) == cls

    A = tf.math.count_nonzero(true & pred) * 2
    B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)

    return tf.math.divide_no_nan(tf.cast(A, tf.float32), tf.cast(B, tf.float32))

In [48]:
def compile_model(training, y, outputs, use_dsc=False, **kwargs):

    sce = losses.SparseCategoricalCrossentropy(from_logits=True)(y_true=y, y_pred=outputs)
    acc = metrics.sparse_categorical_accuracy(y_true=y, y_pred=outputs)

    training.add_loss(sce)
    training.add_metric(acc, 'acc')

    if use_dsc:
        dsc = calculate_dsc(y_true=y, y_pred=outputs)
        training.add_metric(dsc, 'dsc')

    training.compile(optimizer=optimizers.Adam(learning_rate=1e-3))

    return training

### Define pretrain models

Use the following cell block to start building your pretraining model using the backbone network.

In [49]:
def pretrain(xs, clusters, backbone=None, epochs=50, batch_size=15, **kwargs):

    inputs, x, y = create_inputs(**kwargs)

    if backbone is None:
        backbone = create_unet()

    outputs = backbone(x)
    outputs = layers.Conv3D(kernel_size=1, filters=(clusters.max() + 1))(outputs)
    
    training = Model(inputs=inputs, outputs=outputs)
    training = compile_model(training, y, outputs)

    training.fit(x={'x': xs['dat'], 'y': clusters}, epochs=epochs, batch_size=batch_size)

    return backbone, training

In [50]:
backbone, training = pretrain(xs, clusters)

Epoch 1/50
27/27 [==============================] - 3s 22ms/step - loss: 1.8777 - acc: 0.2949
Epoch 2/50
27/27 [==============================] - 1s 21ms/step - loss: 1.4549 - acc: 0.4766
Epoch 3/50
27/27 [==============================] - 1s 21ms/step - loss: 1.2261 - acc: 0.5487
Epoch 4/50
27/27 [==============================] - 1s 21ms/step - loss: 1.0471 - acc: 0.6264
Epoch 5/50
27/27 [==============================] - 1s 21ms/step - loss: 0.9082 - acc: 0.6920
Epoch 6/50
27/27 [==============================] - 1s 21ms/step - loss: 0.7849 - acc: 0.7379
Epoch 7/50
27/27 [==============================] - 1s 21ms/step - loss: 0.6929 - acc: 0.7597
Epoch 8/50
27/27 [==============================] - 1s 21ms/step - loss: 0.6339 - acc: 0.7784
Epoch 9/50
27/27 [==============================] - 1s 21ms/step - loss: 0.5799 - acc: 0.7922
Epoch 10/50
27/27 [==============================] - 1s 21ms/step - loss: 0.5563 - acc: 0.7960
Epoch 11/50
27/27 [==============================] - 1s 22m

### Define fine-tune models

Use the following cell block to start building your fine-tuning model using the backbone network.

In [51]:
def finetune(xs, backbone=None, n_training=10, epochs=500, batch_size=15, validation_freq=50, **kwargs): 

    inputs, x, y = create_inputs(**kwargs)

    if backbone is None:
        backbone = create_unet()

    outputs = backbone(x)
    outputs = layers.Conv3D(kernel_size=1, filters=2)(outputs)
    
    training = Model(inputs=inputs, outputs=outputs)
    training = compile_model(training, y, outputs, use_dsc=True)

    training.fit(
        x={'x': xs['dat'][:n_training], 'y': xs['lbl'][:n_training]}, 
        validation_data={'x': xs['dat'][n_training:], 'y': xs['lbl'][n_training:]}, 
        validation_freq=validation_freq,
        batch_size=max(n_training, batch_size),
        epochs=epochs)

    return backbone, training

### Train the model

Use the following cell block to train your deep clustering model.

In [52]:
backbone, training = finetune(xs, backbone=backbone)

Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 0.7388 - acc: 0.5377 - dsc: 0.2248
Epoch 2/500
1/1 [==============================] - 0s 26ms/step - loss: 0.6310 - acc: 0.6147 - dsc: 0.3414
Epoch 3/500
1/1 [==============================] - 0s 24ms/step - loss: 0.5022 - acc: 0.7604 - dsc: 0.4892
Epoch 4/500
1/1 [==============================] - 0s 23ms/step - loss: 0.4152 - acc: 0.8710 - dsc: 0.6715
Epoch 5/500
1/1 [==============================] - 0s 23ms/step - loss: 0.3581 - acc: 0.9286 - dsc: 0.8003
Epoch 6/500
1/1 [==============================] - 0s 24ms/step - loss: 0.3192 - acc: 0.9491 - dsc: 0.8502
Epoch 7/500
1/1 [==============================] - 0s 23ms/step - loss: 0.2728 - acc: 0.9607 - dsc: 0.8781
Epoch 8/500
1/1 [==============================] - 0s 22ms/step - loss: 0.2492 - acc: 0.9629 - dsc: 0.8786
Epoch 9/500
1/1 [==============================] - 0s 22ms/step - loss: 0.2208 - acc: 0.9691 - dsc: 0.9031
Epoch 10/500
1/1 [=====================

In [57]:
def run_experiment(xs, n_clusters=8, epochs=3, pretrain_epochs=50, finetune_epochs=500, n_training=10):

    backbone = None
    
    for epoch in range(epochs):

        print('==================================================================')
        print('STARTING EPOCH: {}'.format(epoch + 1))
        print('==================================================================')

        kwargs = {
            'backbone': backbone, 
            'x_weight': 1. if epoch == 0 else 0, 
            'coords_weight': 1. if epoch == 0 else 0}

        kmeans, clusters = create_clusters(x=xs['dat'], n_clusters=n_clusters, **kwargs)

        backbone, training = pretrain(xs=xs, clusters=clusters, backbone=backbone, epochs=pretrain_epochs)

    backbone, training = finetune(xs=xs, backbone=backbone, n_training=n_training, epochs=finetune_epochs)
    
    return backbone, training

In [58]:
backbone, training = run_experiment(xs)

STARTING EPOCH: 1
Epoch 1/50
27/27 [==============================] - 3s 22ms/step - loss: 1.8487 - acc: 0.2686
Epoch 2/50
27/27 [==============================] - 1s 21ms/step - loss: 1.4467 - acc: 0.3970
Epoch 3/50
27/27 [==============================] - 1s 21ms/step - loss: 1.2205 - acc: 0.5097
Epoch 4/50
27/27 [==============================] - 1s 21ms/step - loss: 1.0331 - acc: 0.5900
Epoch 5/50
27/27 [==============================] - 1s 21ms/step - loss: 0.9363 - acc: 0.6400
Epoch 6/50
27/27 [==============================] - 1s 21ms/step - loss: 0.8245 - acc: 0.7074
Epoch 7/50
27/27 [==============================] - 1s 21ms/step - loss: 0.7232 - acc: 0.7518
Epoch 8/50
27/27 [==============================] - 1s 21ms/step - loss: 0.6570 - acc: 0.7722
Epoch 9/50
27/27 [==============================] - 1s 21ms/step - loss: 0.5952 - acc: 0.7931
Epoch 10/50
27/27 [==============================] - 1s 21ms/step - loss: 0.5799 - acc: 0.7952
Epoch 11/50
27/27 [======================

# Evaluation

Based on the tutorial discussion, use the following cells to calculate model performance. The following metrics should be calculated:

* Dice score coefficient (mean, median, 25th percentile, 75th percentile)

### Performance

The following minimum performance metrics must be met for full credit:

* median Dice score coefficient: >0.80

In [59]:
n_training = 10
logits = training.predict({'x': xs['dat'], 'y': xs['lbl']})

dice = []
for y_true, y_pred in zip(xs['lbl'][n_training:], logits[n_training:]):
    
    dsc = calculate_dsc(y_true=y_true, y_pred=y_pred).numpy()
    dice.append(dsc)

In [60]:
len(dice)

391

In [65]:
df = pd.DataFrame(np.arange(len(dice)))
df['dice'] = dice


In [68]:
(df['dice'].sum())/len(df['dice'])

0.9365999265704923

In [78]:
df_coef = pd.DataFrame(np.arange(1))
df_coef['mean'] = (df['dice'].sum())/len(df['dice'])
df_coef['median'] = (df['dice'].median())
df_coef['25th'] = (df['dice'].quantile(q= 0.25))
df_coef['75th'] = (df['dice'].quantile(q= 0.75))

In [81]:
df_coef = df_coef[['mean', 'median', '25th', '75th']]

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort sensitivity and Dice score statistics. There is no need to submit training performance accuracy.

In [84]:
df_coef.to_csv('wjhan_results.csv')

# Submission

Use the following line to save your model for submission:

In [85]:
# --- Serialize a model
training.save('./wjhan_model.hdf5')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.